1 - Leitura e Gravação dos dados no Banco SQL - Pesquisa Tweet:

In [255]:
# Imports:
from twython import Twython
import json
import pandas as pd
import time
import pyodbc 

In [256]:
# Conexão com o Banco de Dados - SQL Server:
conn = pyodbc.connect(
    'DRIVER={SQL Server};'
    'SERVER=DESKTOP-2TEIT25\SQLSERVERLOCAL;'
    'DATABASE=DB_TELEMEDICINA;'
    'UID=adm;'
    'PWD=admin')

# Abrir conexão com o banco: 
cursor = conn.cursor()

# Realizamos um truncate na tabela para gravar os dados mais atualizados:
sql_command = ("TRUNCATE TABLE dbo.TB_TWEETS")

# Executa comando Truncate:
cursor.execute(sql_command)

# Criamos um comando de Insert dos dados:
sql_command = ("INSERT INTO dbo.TB_TWEETS ([USUARIO],[DATA],[HORARIO],[POSTAGEM]) VALUES (?,?,?,?)")


In [257]:
# Extraindo as credenciais de acesso de um arquivo .json:
with open("twitter_credentials.json", "r") as file:  
    creds = json.load(file)

In [258]:
# Instanciando um objeto twython:
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])

In [259]:
# Criando query com os parametros a serem passados no método:
query = {'q': 'telemedicina','count':100,'lang': 'pt'}

In [260]:
# Dicionário para armazenar os tweets:
dict_twitter = {'usuario': [], 'data': [], 'horario':[], 'postagem': []}

In [261]:
lista_twitter = []

In [262]:
# Pesquisando o termo "telemedicina" e retornando os 1000 tweets: 
for status in python_tweets.search(**query)['statuses']:  
    usuario = (status['user']['screen_name'])
    data = (time.strftime('%Y-%m-%d', time.strptime(status['created_at'],'%a %b %d %H:%M:%S +0000 %Y')))
    horario = (time.strftime('%H:%M:%S', time.strptime(status['created_at'],'%a %b %d %H:%M:%S +0000 %Y')))
    postagem = (status['text'])
    
    values = [usuario, data, horario, postagem]
    lista_twitter.append(values)
    
    cursor.execute(sql_command, usuario, data, horario, postagem)

In [263]:
# Commit dos dados incluídos no banco:
conn.commit()

-------

2 - Leitura e Exibição dos dados do Banco SQL - Pesquisa Tweet:

In [264]:
# Comando SQL para extrair os dados dos usuários com mais tweets:
sql_command_link = """
    SELECT 
        COUNT(*) AS QTD_TWEET, 
        USUARIO
    FROM [dbo].[TB_TWEETS]
    GROUP BY USUARIO
    ORDER BY QTD_TWEET DESC
"""

In [265]:
# Executa comando SQL_Link:
return_query_link = cursor.execute(sql_command_link).fetchall()

In [266]:
# Converter lista de registros em um Dataframe:
dfQtdeTweet = pd.DataFrame.from_records(return_query_link,columns=['Quantidade_Tweet','Usuario'])

In [267]:
# Imprime o dataFrame referente aos dados de Domínio:
print(dfQtdeTweet.head(20))

    Quantidade_Tweet          Usuario
0                  4         Hosp_ESS
1                  2  neurocirurgiabr
2                  2           sebrae
3                  2    RochaRosylane
4                  2   telessaudeuerj
5                  2   1berto_almeida
6                  2  vladimirpezzole
7                  2  wagnervilasboas
8                  2        motonauer
9                  1      multilaudos
10                 1      nanderson14
11                 1           _lalas
12                 1          WalsirT
13                 1    web_alexandre
14                 1    WeNeuroTweets
15                 1   AcademiaMedica
16                 1        alexappel
17                 1         ALLTECH5
18                 1         AnapsiDE
19                 1     andre_kadosh


-------

In [268]:
# Comando SQL para extrair os dados dos usuários com mais tweets:
sql_command_link = """
SELECT 
    COUNT(*) AS QTD_TWEET,
    CASE DATEPART(dw,[DATA])
        WHEN (1) THEN 'Domingo'
        WHEN (2) THEN 'Segunda-Feira'
        WHEN (3) THEN 'Terça-Feira'
        WHEN (4) THEN 'Quarta-Feira'
        WHEN (5) THEN 'Quinta-Feira'
        WHEN (6) THEN 'Sexta-Feira'
        WHEN (7) THEN 'Sabado'
    END DIA_DA_SEMANA
FROM dbo.TB_TWEETS
GROUP BY
    CASE DATEPART(dw,[DATA])
        WHEN (1) THEN 'Domingo'
        WHEN (2) THEN 'Segunda-Feira'
        WHEN (3) THEN 'Terça-Feira'
        WHEN (4) THEN 'Quarta-Feira'
        WHEN (5) THEN 'Quinta-Feira'
        WHEN (6) THEN 'Sexta-Feira'
        WHEN (7) THEN 'Sabado'
    END
ORDER BY QTD_TWEET DESC
"""

In [269]:
# Executa comando SQL_Link:
return_query_link = cursor.execute(sql_command_link).fetchall()

In [270]:
# Converter lista de registros em um Dataframe:
dfDiaSemana = pd.DataFrame.from_records(return_query_link,columns=['Quantidade_Tweet','Dia_Semana'])

In [271]:
# Imprime o dataFrame:
print(dfDiaSemana)

   Quantidade_Tweet     Dia_Semana
0                23   Quinta-Feira
1                23  Segunda-Feira
2                16        Domingo
3                12    Terça-Feira
4                11    Sexta-Feira
5                 9         Sabado
6                 6   Quarta-Feira


-------

In [272]:
# Comando SQL para extrair os dados dos usuários com mais tweets:
sql_command_link = """
    SELECT 
        COUNT(*) AS QTD_TWEET,
        DATEPART(HOUR,HORARIO) AS HORA_DIA
    FROM TB_TWEETS
    GROUP BY DATEPART(HOUR,HORARIO)
    ORDER BY QTD_TWEET DESC
"""

In [273]:
# Executa comando SQL_Link:
return_query_link = cursor.execute(sql_command_link).fetchall()

In [274]:
# Converter lista de registros em um Dataframe:
dfHora = pd.DataFrame.from_records(return_query_link,columns=['Quantidade_Tweet','Hora_Dia'])

In [275]:
# Imprime o dataFrame:
print(dfHora)

    Quantidade_Tweet  Hora_Dia
0                 14        20
1                 11        19
2                 10        23
3                 10        10
4                  7        14
5                  6        15
6                  6        21
7                  6         0
8                  4         1
9                  4        16
10                 4        17
11                 3        22
12                 3        12
13                 3        13
14                 2         2
15                 2         3
16                 2         9
17                 2        18
18                 1        11


In [276]:
cursor.close()
conn.close()